In [1]:
import boto3
import pandas as pd
import mlflow
import mlflow.sklearn
import os
import mlflow.sklearn

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp
import mlflow

In [2]:
# Инициализация клиента
print('Инициализация клиента...')
s3 = boto3.client('s3',
                  endpoint_url='http://localhost:9000',
                  aws_access_key_id='minio',
                  aws_secret_access_key='minio123')

Инициализация клиента...


In [3]:
# Считывание данных
print('Считывание данных...')
obj = s3.get_object(Bucket='datasets', Key='kinopoisk_train.csv')
data = obj['Body'].read().decode('utf-8')
df = pd.read_csv(StringIO(data))

Считывание данных...


In [4]:
# Установка переменных окружения в Windows
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5000'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio123'

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2)

# Определение пространства поиска
space = {
    'preprocessing': hp.choice('preprocessing', [None, 'remove_stopwords']),
    'vectorizer': hp.choice('vectorizer', ['CountVectorizer', 'TfidfVectorizer']),
    'model': hp.choice('model', ['LogisticRegression', 'RandomForest']),
    'C': hp.uniform('C', 0.1, 10.0)
}

In [6]:
def objective(params):
    # Извлечение параметров из пространства поиска
    preprocessing = params['preprocessing']
    vectorizer_type = params['vectorizer']
    model_type = params['model']
    C = params['C']

   # Применение выбранных методов предобработки данных, векторизации и модели классификации
    if preprocessing == 'remove_stopwords':
        pass

    if vectorizer_type == 'CountVectorizer':
        vectorizer = CountVectorizer()
    elif vectorizer_type == 'TfidfVectorizer':
        vectorizer = TfidfVectorizer()

    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    if model_type == 'LogisticRegression':
        clf = LogisticRegression(C=C)
    elif model_type == 'RandomForest':
        clf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Обучение модели и вычисление метрик
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    accuracy = accuracy_score(y_test, y_pred)
    print('Точность модели:', accuracy)

    # Логирование в MLflow
    with mlflow.start_run() as run:
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(clf, "model", registered_model_name="OptimizedModel")

    # Возвращение целевой метрики, которую Hyperopt будет оптимизировать
    return -accuracy

# Оптимизация гиперпараметров с использованием Hyperopt
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=25)

Точность модели:                                                                                                       
0.7099468488990129                                                                                                     
  0%|                                                                           | 0/25 [01:11<?, ?trial/s, best loss=?]

C:\Users\Huawei\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")

Registered model 'OptimizedModel' already exists. Creating a new version of this model...
2024/01/21 14:12:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: OptimizedModel, version 44



  4%|█▉                                              | 1/25 [01:20<32:03, 80.17s/trial, best loss: -0.7099468488990129]

Created version '44' of model 'OptimizedModel'.


Точность модели:                                                                                                       
0.7757529739306505                                                                                                     
  4%|█▉                                              | 1/25 [01:45<32:03, 80.17s/trial, best loss: -0.7099468488990129]

...

Created version '67' of model 'OptimizedModel'.


Точность модели:                                                                                                       
0.7772715768159959                                                                                                     
 96%|█████████████████████████████████████████████  | 24/25 [19:43<00:34, 34.53s/trial, best loss: -0.7777777777777778]

C:\Users\Huawei\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

Registered model 'OptimizedModel' already exists. Creating a new version of this model...
2024/01/21 14:31:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: OptimizedModel, version 68



100%|███████████████████████████████████████████████| 25/25 [19:48<00:00, 47.53s/trial, best loss: -0.7777777777777778]


Created version '68' of model 'OptimizedModel'.
